In [15]:
import sys
sys.path.append("/Users/marchan/Documents/GitHub/PORTFOLIO_PROJECT/LIBRARIES/ThinkDSP/code")
sys.path.append("/Users/marchan/Documents/GitHub/PORTFOLIO_PROJECT/LIBRARIES/")
sys.path.append("Users/luigibungaro/dsr/Audio_Probreat/")

In [8]:
from pyAudioAnalysis import audioFeatureExtraction as aFE, audioBasicIO,\
audioAnalysis,audioSegmentation,audioVisualization,audioTrainTest
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import numpy as np
import keras
from keras import models,layers
import os
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, GRU
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [10]:
def amplitude_data_all():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                data.append(X)
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                data.append(X)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                data.append(X)
                target.append(['2','extrahls'])
          
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    sampling_indexes = np.arange(0,threshold,10)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
    data = np.array(data)

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [11]:
def amplitude_data_sample_mean(step_of_sampling):
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                data.append(X)
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                data.append(X)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                data.append(X)
                target.append(['2','extrahls'])
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
    data = pd.DataFrame(data)
    data.rolling(window = step_of_sampling, axis = 1).mean().iloc[:,range(step_of_sampling-1,len(data[0]),step_of_sampling)]
    #data.groupby(lambda x: x // 10, axis = 1).mean()
    data = np.array(data)
    data = data[:,0::step_of_sampling]
    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [12]:
def amplitude_data_sample_asis(step_of_sampling):
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                data.append(X)
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                data.append(X)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                data.append(X)
                target.append(['2','extrahls'])
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    #sampling_indexes = np.arange(0,threshold,step_of_sampling)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
        
    data = np.array(data)
    data = data[:,0::step_of_sampling]

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [13]:
def amplitude_data_difference():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                X_temp = X[1:]
                data.append(X_temp-X[:-1])
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                #X_temp = X[1:]
                #data.append(X_temp-X[:-1])
                data.append(data)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                X_temp = X[1:]
                data.append(X_temp-X[:-1])
                target.append(['2','extrahls'])
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    sampling_indexes = np.arange(0,threshold,10)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
        #data[k]=i[sampling_indexes]
    data = np.array(data)

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [14]:
X_train,X_test,y_train,y_test = amplitude_data_all()

FileNotFoundError: [Errno 2] No such file or directory: 'peterbertley_Audio/Atraining_murmur'

In [61]:
X_train,X_test,y_train,y_test = amplitude_data_sample_asis(50)

shape before dropping: 84 84
shape after dropping: 76 76
final shape of your data is:  (76, 4410)
final categorical target shape is: (76, 3)


/Users/luigibungaro/dsr/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [65]:
X_train,X_test,y_train,y_test = amplitude_data_sample_mean(50)

shape before dropping: 84 84
shape after dropping: 76 76
final shape of your data is:  (76, 4410)
final categorical target shape is: (76, 3)


/Users/luigibungaro/dsr/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [63]:
X_train,X_test,y_train,y_test = amplitude_data_difference()

shape before dropping: 84 84
shape after dropping: 47 47


/Users/luigibungaro/dsr/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


final shape of your data is:  (47, 220500)
final categorical target shape is: (47, 3)
